# Тестирование in-context learning для Mistral-7B-Instruct-v0.2

In [1]:
from vllm import LLM, SamplingParams
from tqdm import tqdm

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

INFO 04-07 18:37:27 llm_engine.py:70] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 04-07 18:37:38 llm_engine.py:275] # GPU blocks: 12690, # CPU blocks: 2048
INFO 04-07 18:37:39 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-07 18:37:39 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 04-07 18:37:41 model_runner.py:547] Graph capturing finished in 3 se

In [2]:
import pandas as pd
import json

df = pd.read_csv('Data/generated_data.csv')
df.json = df.json.apply(lambda x: eval(x))

In [3]:
from utils import *
ner = NER()

In [4]:
sampling_params = SamplingParams(temperature=0, max_tokens=100, stop='}', include_stop_str_in_output=True)

for i in tqdm(range(2000)):
    sample = df.iloc[i]
    prompt = f"""Ты должен найти в тексте определенные атрибуты и сохранить их в формате JSON. Не используй комментарии.
    
    Используй следующий шаблон:
    
    Текст: сообщение пользователя в чате.
    JSON: объект по всем стандартам JSON.{ner.label_mapping[sample.label]['template']}
    Начинай!
    
    Текст: {sample.text}
    JSON:"""
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    ner.count_errors(sample, generated_text)

100%|██████████| 2000/2000 [56:59<00:00,  1.71s/it]


In [5]:
ner.score()

Точность для класса "самолет"

	departure_city: 0.77300

	arrival_city: 0.89900

	departure_date: 0.80300

	return_date: 0.88500

Точность для класса "отель"

	city: 0.49300

	hotel: 0.71400

	date: 0.28500

	guests: 0.70800

	days: 0.48800



__Вывод:__
Модель с трудом определяет верные даты и числа, однако с названиями городов и отелей справляется хорошо.

__Дальнейшие шаги:__
Необходимо дообучить модель на задачу NER используя 80% данных. Предварительно нужно замерить точность на тестовой выборке, чтобы сравнить улучшение результата после тренировки.

# Точность на тестовых данных

In [4]:
sampling_params = SamplingParams(temperature=0, max_tokens=100, stop='}', include_stop_str_in_output=True)

In [5]:
test_ner = NER()

In [6]:
test_df = pd.concat([df.iloc[800:1000], df.iloc[1800:2000]])

In [7]:
for i in tqdm(range(400)):
    sample = test_df.iloc[i]
    prompt = f"""Ты должен найти в тексте определенные атрибуты и сохранить их в формате JSON. Не используй комментарии.

Используй следующий шаблон:

Текст: сообщение пользователя в чате.
JSON: объект по всем стандартам JSON.{test_ner.label_mapping[sample.label]['template']}
Начинай!

Текст: {sample.text}
Ответ:"""
    
    output = llm.generate(prompt, sampling_params, use_tqdm=False)[0]
        
    # Print the outputs.
    prompt = output.prompt
    generated_text = output.outputs[0].text
    test_ner.count_errors(sample, generated_text)

100%|██████████| 400/400 [11:24<00:00,  1.71s/it]


In [8]:
test_ner.score()

Точность для класса "самолет"

	departure_city: 0.78500

	arrival_city: 0.90000

	departure_date: 0.79000

	return_date: 0.86500

Точность для класса "отель"

	city: 0.47000

	hotel: 0.80500

	date: 0.27000

	guests: 0.82000

	days: 0.51000

